In [2]:
library(GWENA)
library(magrittr) 

In [3]:
exprs <- read.delim(snakemake@input$vst, sep="\t", as.is=TRUE, check.names=FALSE, row.names=1)
exprs <- exprs[rowMeans(exprs)>5,]

In [4]:
sample.info <- read.delim("data/tmp/sample_info.tsv", sep="\t", row.names=1)
sample.info <- sample.info[colnames(exprs),]
gene.info <- read.delim("data/tmp/rnaseq/quant/salmon/tximport/gene_info.tsv", sep="\t", row.names=1, as.is=TRUE)
gene.info <- gene.info[rownames(exprs),]

In [5]:
head(exprs)

In [6]:
head(sample.info)

In [7]:
head(gene.info)

In [8]:
se <- SummarizedExperiment::SummarizedExperiment(assays=list(expr=exprs), colData=sample.info, rowData=gene.info)
dim(se)

**Filter expression data**

In [11]:

se_filt <- filter_low_var(se, pct = 0.7, type = "median")

In [12]:
dim(se_filt)

In [26]:
?build_net

In [30]:
net <- build_net(se_filt, cor_func = "spearman", n_threads = 24)

In [31]:
net$metadata

In [33]:
plot(net$metadata$fit_power_table[,2])

In [34]:
fit_power_table <- net$metadata$fit_power_table
fit_power_table[fit_power_table$Power == net$metadata$power, "SFT.R.sq"]

In [32]:
?detect_modules

In [35]:
modules <- detect_modules(se_filt, net$network, detailled_result = TRUE, merge_threshold = 0.75)

In [36]:
layout_mod_merge <- plot_modules_merge(modules_premerge = modules$modules_premerge, modules_merged = modules$modules)

In [37]:
ggplot2::ggplot(data.frame(modules$modules %>% stack), 
                ggplot2::aes(x = ind)) + ggplot2::stat_count() +
  ggplot2::ylab("Number of genes") +
  ggplot2::xlab("Module")

In [41]:
plot_expression_profiles(se_filt, modules$modules)

In [38]:
enrichment <- bio_enrich(modules$modules)
plot_enrichment(enrichment)

In [51]:
eigengenes <- mm
all(lapply(eigengenes, function(x) is.character(x) | is.numeric(x) | is.logical(x)) %>% unlist)

In [42]:
ss <- sample.info %>% dplyr::select(Sample_Group)
ss$Sample_Group <- factor(ss$Sample_Group)
ss$Sample_Group <- relevel(ss$Sample_Group, ref = "ctr")
ss <- cbind(ss, ss)
mm <- as.matrix(modules$modules_eigengenes)
phenotype_association <- associate_phenotype(eigengenes=mm, phenotypes=ss)

In [44]:
GWENA::associate_phenotype

In [43]:
??associate_phenotype

In [90]:
module_example <- modules$modules$`2`
graph <- build_graph_from_sq_mat(net$network[module_example, module_example])

In [91]:
layout_mod_2 <- plot_module(graph, upper_weight_th = 0.999995, 
                            vertex.label.cex = 0, 
                            node_scaling_max = 7, 
                            legend_cex = 1)